In [ ]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!wget -c "https://figshare.com/ndownloader/files/10798046"

In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')

messages = pd.read_csv("super23.csv", encoding='latin-1')
messages.columns = ["label","message"]
print (messages.head())

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map
embeddings = read_glove_vector('glove.6B.100d.txt') #glove.6B.50d.txt

In [ ]:
url = "GoogleNews-vectors-negative300.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [ ]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in messages['message'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
print (docs_vectors.shape)

(65736, 100)


In [ ]:
docs_vectors['label'] = messages['label']
docs_vectors = docs_vectors.dropna()

# docs_vectors_train = docs_vectors.loc[0:55460, :]
docs_vectors_train = docs_vectors.loc[37816:65736, :]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

train_x, test_x, train_y, test_y = train_test_split(docs_vectors_train.drop('label', axis = 1),
                                                   docs_vectors_train['label'],
                                                   test_size = 1,
                                                   random_state = 1)
print (train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(27900, 100) (27900,) (1, 100) (1,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

model = SVC()
# model = SVC(gamma='scale', random_state=1)
model.fit(train_x, train_y)

SVC()

In [ ]:
from sklearn import metrics
docs_vectors_test = docs_vectors.loc[0:37815, :] # 55461:55685 55686:55910 55911:56135 56136:56360 56361:56585 56586:56810
labels_test = docs_vectors_test.iloc[: , -1]
docs_vectors_test = docs_vectors_test.iloc[:,:-1]
# print ("test vects", docs_vectors_test)
# print ("test labels", labels_test)

test_pred = model.predict(docs_vectors_test)
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(labels_test, test_pred))
# print(metrics.classification_report(labels_test, test_pred, digits=4))
print('Precision: ', metrics.precision_score(labels_test, test_pred))
print('Recall: ', metrics.recall_score(labels_test, test_pred))
from sklearn.metrics import accuracy_score
print ("Accuracy: ",accuracy_score(labels_test, test_pred))